In [1]:
### Baseline recommender - priporoča 15 najbolj prodajanih artiklov prejšnjega tedena (v skupnem)###
# napovedujem samo nakupe v tednu 2018-06-27 do 2018-07-04

In [2]:
#importi
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from datetime import date, timedelta
from collections import Counter

In [3]:
parse_dates = ['datum']
print("loading nakupi.csv...")
orders = pd.read_csv("Podatki/nakupi.csv", dtype={ #nrows=500000
        'kupec': np.uint32,
        'st_racuna': np.uint32,
        'artikel': np.uint32,
        'letak': np.bool},
        parse_dates=parse_dates)
print("done")

loading nakupi.csv...
done


In [4]:
#Preparing data sets train/test
print("Preparing data")
max_datum = orders['datum'].max()            

tmp = pd.DataFrame()
orders['last_order'] = orders.groupby('kupec')['st_racuna'].transform(max)
tmp = orders.assign(last=orders.datum >= max_datum - timedelta(7)) #zadnji teden
tmp = tmp.assign( manj_kot_14= tmp.datum >= max_datum - timedelta(14) )
tmp = tmp.assign( vec_kot_7= tmp.datum < max_datum - timedelta(7) )
tmp = tmp.assign( second_last= tmp.manj_kot_14 & tmp.vec_kot_7) #predzadnji teden

### TEST ###
test_second_last = tmp.loc[tmp['second_last'] == True] #  ostanejo samo predzadnji zadnji nakup ali ne
test_second_last = test_second_last.groupby('kupec').agg({'artikel':lambda x: list(x)}) # transforms real artikels to list

test_last = tmp.loc[tmp['last'] == True] # ostanejo samo zadni nakupi
test_for_f1 = test_last.groupby('kupec').agg({'artikel':lambda x: list(x)}) # transforms real artikels to list
print("done")

Preparing data
done


In [5]:
max_datum - timedelta(7)

Timestamp('2018-06-27 00:00:00')

In [6]:
max_datum

Timestamp('2018-07-04 00:00:00')

In [7]:
#naredimo count: dict z pogostostjo nakupa izdelka
all_list = list()
for entry in test_second_last.itertuples():
    all_list += entry[1]

count = Counter(all_list)

In [8]:
#naredimo seznam top 10 izdelkov prejšnjega tedna
sorted_list = sorted(count.items(), key=lambda x: x[1], reverse=True)
top_15 = list()

for i in range(0, 15):
    top_15.append(sorted_list[i][0])
top_15

[2, 16, 440, 1042, 363, 441, 57, 256, 604, 824, 446, 34, 294, 1351, 74]

In [9]:
test_for_f1.head(2) #to so zadnji nakupi (torej nakupi v zadnjem tednu)

,artikel
kupec,
1,"[247, 5438, 195, 4703, 4434, 3221, 5669, 4691,..."
4,"[227, 1351, 9191, 322, 4201]"


In [10]:
### TESTING ####

def eval_fun(labels, preds):
    labels = labels
    preds = preds#.split(' ')
    rr = (np.intersect1d(labels, preds))
    if isinstance(preds, float) :
        return (0,0,0)
    precision = np.float(len(rr)) / len(preds)
    recall = np.float(len(rr)) / len(labels)
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)


###########
print("done")
len(test_for_f1)

done


14771

In [11]:
res = list()
for entry in test_for_f1.itertuples():
    #print("order:{} preds:{}".format(entry[1], entry[2]))
    res.append(eval_fun(entry[1], top_15))

res = pd.DataFrame(np.array(res), columns=['precision', 'recall', 'f1'])

In [12]:
res.describe()

,precision,recall,f1
count,14771.000000,14771.000000,14771.000000
mean,0.109968,0.091009,0.084659
std,0.105051,0.103576,0.072244
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.066667,0.071429,0.081081
75%,0.200000,0.125000,0.129032
max,0.733333,1.000000,0.476190
